In [ ]:
#  For project clustering steps:
# 1- Load the Pandas data frame songs of the audio features (at least 1000 songs as different as possible).🎹 ----DONE
# 2- Standardize the data using Standardscaler ----DONE
# 3- Save the scaler for future use on the new user input song, using Pickle. (tomorrow)
# 4- Select the number of clusters k.
# 5- Adjust k means cluster.
# 6- Build the elbow graph to find the best k. (tomorrow)
# 7- Use the model with the best k to assign each observation in your data frame to its cluster number using model.predict (add cluster columns to the pandas data frame).
# 8- Save the model with the best k as your final model using pickle. (tomorrow)

# 📍 For the final product tasks and the full scenario:
# 1- Get the song name from the user as input ---------- DONE
# 2- Play the input song in the music embed player. ---------- DONE
# 3- Get the audio features for this song from the Spotify API using sp.audio_features(trackid). Be careful to keep only the audio features columns.---------- DONE
# 4- Load the StandardScaler using Pickle and use it to scale the REQUESTED_SONG.(tomorrow)
# 5- Use kmeans_model.predict(new scaled audio dataset for the REQUESTED_SONG) to predict the cluster (label) for the REQUESTED_SONG.
# 6- Return a random song from the same cluster that the REQUESTED_SONG belongs to from your data frame and suggest it to the user.
# 7- Play it using the built-in music player.
# 8*- Try to Streamlit? ☀️ (optional) 

In [ ]:
from stackapi import StackAPI
from bs4 import BeautifulSoup
import requests as re

import pandas as pd
import numpy as np

from random import randint    
from datetime import datetime, timedelta
from time import sleep

import json
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pprint

client_id = os.environ.get("SPOTIPY_CLIENT_ID")
client_secret = os.environ.get("SPOTIPY_CLIENT_SECRET")

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

from plotly import express as px
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt

# ***************************** PART I *****************************

## Loading our Songs Database

In [ ]:
df = pd.read_csv('Songs_Database.csv', sep=';')
# df.dropna(subset=["id"], inplace=True)
df

In [ ]:
data = df.drop(["id"], axis=1)
data.head()

## Performing Scaling on our Songs_Database

In [ ]:
# Create an instance of StandardScaler
scaler = StandardScaler()

# Standardize the data using StandardScaler
X_normalized = scaler.fit_transform(data[["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness","liveness","valence","tempo","duration_ms"]])

# Print the original data
print("Original data:")
# print(data.head().to_frame())
display(data.head())

# Create a DataFrame with the normalized data
data_normalized = pd.DataFrame(X_normalized, columns=["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness","liveness","valence","tempo","duration_ms"])

# Print the scaled data
print("\nScaled data:")
data_normalized.head()

In [ ]:
# Perform PCA on the normalized data
pca = PCA(n_components=10)
pca.fit(X_normalized)

# Get the contribution of each principal component to the data variance
# En utilisant pca.explained_variance_ratio_, on peut obtenir une idée de la contribution de chaque composante principale à la variance des données.
# Cela peut aider à décider combien de composantes principales utiliser pour réduire la dimensionnalité ou pour d'autres analyses.
pca.explained_variance_ratio_

# Get the contribution of each principal component to the data variance
explained_variance_ratio = pca.explained_variance_ratio_

# Print the contribution of each principal component
print("Explained variance ratio:")
print(explained_variance_ratio)

# Calculate the cumulative sum of explained variance ratio
# Déterminer combien de composantes principales tu souhaites conserver, en fonction du pourcentage de variance que tu souhaites préserver.
# Par exemple, si la somme cumulative atteint 0,95, cela signifie que les composantes principales retenues capturent 95 % de la variance totale des données.
np.cumsum(pca.explained_variance_ratio_)

# Calculate the cumulative sum of explained variance ratio
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

# Print the cumulative sum of explained variance ratio
print("\nCumulative variance ratio:")
print(cumulative_variance_ratio)

## Save SCALER in file with PICKLE

In [ ]:
import pickle

#Ici, nous ouvrons un fichier nommé "scaler.pkl" en mode d'écriture binaire ('wb'), puis nous utilisons pickle.dump() pour écrire l'objet scaler dans le fichier.
with open('scaler.pkl', 'wb') as fichier:
    pickle.dump(scaler, fichier)

In [ ]:
# with open("Songs_Database.csv", "rb") as f: #don't forget the correct path
#      scaler_new = pickle.load(fichier) 

## Finding out the best number of Clusters to use

In [ ]:
K = range(5, 30) #range of the clusters
inertia = []

for k in K: #for every data in set build a inertia
    print("Training a K-Means model with {} clusters! ".format(k))
    print()
    kmeans = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans.fit(data)
    inertia.append(kmeans.inertia_)

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,8))
plt.plot(K, inertia, 'bx-')
plt.xlabel('k')
plt.ylabel('inertia')
plt.xticks(np.arange(min(K), max(K)+1, 1.0)) #changing the scale
plt.title('Elbow Method showing the optimal k') 

## PCA and KMeans

In [ ]:
# Transform the normalized data using the principal components
X_pca = pca.transform(X_normalized)

# Apply K-means clustering to the transformed data
kmeans = KMeans(n_clusters=14, n_init=12)
kmeans.fit(X_pca)
cluster_ids = kmeans.predict(X_pca)
cluster_distances = kmeans.transform(X_pca)

# Print the cluster IDs and distances
print("\nCluster IDs:")
print(cluster_ids)
print("\nCluster distances:")
print(cluster_distances)

## Assigning a cluster to each song

In [ ]:
data_normalized['cluster'] = cluster_ids
data_normalized

In [ ]:
data.head()

In [ ]:
data_normalized.reset_index(drop=True, inplace=True)  # Restaurer l'index d'origine
column = df.loc[:, "id"]
data_normalized["id"] = column

data_normalized

## 8- Saving model with best K as final model using Pickle

In [ ]:
meilleur_k = 14  # Remplacez par le meilleur k que vous avez sélectionné
kmeans_final = KMeans(n_clusters=meilleur_k)

kmeans_final.fit(X_pca)  # Remplacez X_pca par vos données normalisées

# Définir le nom du fichier pour sauvegarder le modèle
nom_fichier_modele = "k_means_final_model.pkl"

# Sauvegarder le modèle avec Pickle
with open(nom_fichier_modele, 'wb') as fichier:
    pickle.dump(kmeans_final, fichier)

In [ ]:
#Obtenir les identifiants uniques de cluster dans cluster_ids,
#puis il itère sur ces identifiants pour tracer un nuage de points pour chaque cluster.

for c in np.unique(cluster_ids):
    plt.scatter(X_pca[cluster_ids == c, 0], X_pca[cluster_ids == c, 1], marker='.', s=50)

In [ ]:
#Utilisation de fit_predict de KMeans pour effectuer à la fois l'ajustement du modèle K-means 
# et la prédiction des clusters pour les données normalisées X_normalized.
cluster_ids = KMeans(n_clusters=9, n_init=100).fit_predict(X_normalized)

for c in np.unique(cluster_ids):
    plt.scatter(X_normalized[cluster_ids == c, 0], X_normalized[cluster_ids == c, 1], marker='.', s=50)

plt.show()

In [ ]:
#Utilisation de fit_predict de KMeans pour effectuer à la fois l'ajustement du modèle K-means 
# et la prédiction des clusters pour les données normalisées X_normalized.

cluster_ids = KMeans(n_clusters=14, n_init=100).fit_predict(X_normalized)

for c in np.unique(cluster_ids):
    plt.scatter(X_normalized[cluster_ids == c, 0], X_normalized[cluster_ids == c, 1], marker='.', s=50)

plt.show()

# ***************************** PART II *****************************

## Getting the song from user

In [239]:
x = str(input("Please enter the name of a song you'd like to listen to: "))

## Creating the JSON

In [240]:
results = sp.search(q= x, limit=1, market="FR")

json_results = json.dumps(results, ensure_ascii=True)
print(json_results)

{"tracks": {"href": "https://api.spotify.com/v1/search?query=Welcome+to+detroy&type=track&market=FR&offset=0&limit=1", "items": [{"album": {"album_type": "album", "artists": [{"external_urls": {"spotify": "https://open.spotify.com/artist/7tT5h21rKj9yFfQuqBYLqT"}, "href": "https://api.spotify.com/v1/artists/7tT5h21rKj9yFfQuqBYLqT", "id": "7tT5h21rKj9yFfQuqBYLqT", "name": "Trick Trick", "type": "artist", "uri": "spotify:artist:7tT5h21rKj9yFfQuqBYLqT"}], "external_urls": {"spotify": "https://open.spotify.com/album/0siKp9welK70TSwKHywPgM"}, "href": "https://api.spotify.com/v1/albums/0siKp9welK70TSwKHywPgM", "id": "0siKp9welK70TSwKHywPgM", "images": [{"height": 640, "url": "https://i.scdn.co/image/ab67616d0000b2738d8e053727a618a608be2f21", "width": 640}, {"height": 300, "url": "https://i.scdn.co/image/ab67616d00001e028d8e053727a618a608be2f21", "width": 300}, {"height": 64, "url": "https://i.scdn.co/image/ab67616d000048518d8e053727a618a608be2f21", "width": 64}], "is_playable": true, "name": 

## Displaying JSON

In [241]:
results['tracks']["items"][0].keys() # All the dictionary
pprint.pprint(results['tracks']['items'][0])

results['tracks']['items'][0]['id'] # Getting song ID

for item in results['tracks']['items']:
    print("The name of song is: '{}' and the id is: {}".format(item['name'],item["id"]))

{'album': {'album_type': 'album',
           'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7tT5h21rKj9yFfQuqBYLqT'},
                        'href': 'https://api.spotify.com/v1/artists/7tT5h21rKj9yFfQuqBYLqT',
                        'id': '7tT5h21rKj9yFfQuqBYLqT',
                        'name': 'Trick Trick',
                        'type': 'artist',
                        'uri': 'spotify:artist:7tT5h21rKj9yFfQuqBYLqT'}],
           'external_urls': {'spotify': 'https://open.spotify.com/album/0siKp9welK70TSwKHywPgM'},
           'href': 'https://api.spotify.com/v1/albums/0siKp9welK70TSwKHywPgM',
           'id': '0siKp9welK70TSwKHywPgM',
           'images': [{'height': 640,
                       'url': 'https://i.scdn.co/image/ab67616d0000b2738d8e053727a618a608be2f21',
                       'width': 640},
                      {'height': 300,
                       'url': 'https://i.scdn.co/image/ab67616d00001e028d8e053727a618a608be2f21',
            

## Understanding the JSON

In [242]:
print("The json file has the following keys: ", list(results.keys())) # We can see that we only have tracks
print("The 'tracks' key has the following child keys: ", list(results["tracks"].keys())) # Let's check the values
print("The query we made is: ", results["tracks"]["href"]) # Query we have searched 
print("The song's info is contained in: ", results["tracks"]["items"]) # items (actual tracks)
print("The limit of the query we've made is: ", results["tracks"]["limit"]) # Limit we have chosen
print("The next page if any: ", results["tracks"]["next"]) # Link to the next page (next 50 tracks)
print("The starting webpage: ", results["tracks"]["offset"]) # Actual offset (starting point)
print("Starting webpage: ", results["tracks"]["previous"]) # Previous search
print("Total number of results: ", results["tracks"]["total"]) # Number of matches

The json file has the following keys:  ['tracks']
The 'tracks' key has the following child keys:  ['href', 'items', 'limit', 'next', 'offset', 'previous', 'total']
The query we made is:  https://api.spotify.com/v1/search?query=Welcome+to+detroy&type=track&market=FR&offset=0&limit=1
The song's info is contained in:  [{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7tT5h21rKj9yFfQuqBYLqT'}, 'href': 'https://api.spotify.com/v1/artists/7tT5h21rKj9yFfQuqBYLqT', 'id': '7tT5h21rKj9yFfQuqBYLqT', 'name': 'Trick Trick', 'type': 'artist', 'uri': 'spotify:artist:7tT5h21rKj9yFfQuqBYLqT'}], 'external_urls': {'spotify': 'https://open.spotify.com/album/0siKp9welK70TSwKHywPgM'}, 'href': 'https://api.spotify.com/v1/albums/0siKp9welK70TSwKHywPgM', 'id': '0siKp9welK70TSwKHywPgM', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b2738d8e053727a618a608be2f21', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab6

## Other Infos

In [243]:
print("Track infos")
print(results["tracks"]["items"][0]["artists"]) # Track artists

# print("Track artists")
# print(results["tracks"]["items"][0]["artists"]["0"]["name"]) # Track artists



print("\nTrack ID")
print(results["tracks"]["items"][0]["id"]) # Track ID
print("\nTrack name")
print(results["tracks"]["items"][0]["name"]) # Track name
print("\nPopularity index")
print(results["tracks"]["items"][0]["popularity"]) # Popularity index
print("\nBasically ID")
print(results["tracks"]["items"][0]["uri"]) # Basically ID


Track infos
[{'external_urls': {'spotify': 'https://open.spotify.com/artist/7tT5h21rKj9yFfQuqBYLqT'}, 'href': 'https://api.spotify.com/v1/artists/7tT5h21rKj9yFfQuqBYLqT', 'id': '7tT5h21rKj9yFfQuqBYLqT', 'name': 'Trick Trick', 'type': 'artist', 'uri': 'spotify:artist:7tT5h21rKj9yFfQuqBYLqT'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/7dGJo4pcD2V6oG8kP0tJRR'}, 'href': 'https://api.spotify.com/v1/artists/7dGJo4pcD2V6oG8kP0tJRR', 'id': '7dGJo4pcD2V6oG8kP0tJRR', 'name': 'Eminem', 'type': 'artist', 'uri': 'spotify:artist:7dGJo4pcD2V6oG8kP0tJRR'}]

Track ID
0ElQWopuhQ10wgGouWSVlR

Track name
Welcome to Detroit (feat. Eminem)

Popularity index
44

Basically ID
spotify:track:0ElQWopuhQ10wgGouWSVlR


## Playing the requested Song

In [244]:
from IPython.display import IFrame

track_id = results["tracks"]["items"][0]["id"]

def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id+"?autoplay=1",
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [245]:
play_song(track_id)

## Track ID of the Song

In [246]:
song_id = results["tracks"]["items"][0]["id"]
song_id

'0ElQWopuhQ10wgGouWSVlR'

## Getting the audio features of the Requested Song

In [247]:
print(sp.audio_features(song_id))
sp.audio_features(song_id)

[{'danceability': 0.588, 'energy': 0.869, 'key': 6, 'loudness': -1.998, 'mode': 1, 'speechiness': 0.322, 'acousticness': 0.22, 'instrumentalness': 0, 'liveness': 0.585, 'valence': 0.529, 'tempo': 91.128, 'type': 'audio_features', 'id': '0ElQWopuhQ10wgGouWSVlR', 'uri': 'spotify:track:0ElQWopuhQ10wgGouWSVlR', 'track_href': 'https://api.spotify.com/v1/tracks/0ElQWopuhQ10wgGouWSVlR', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0ElQWopuhQ10wgGouWSVlR', 'duration_ms': 286904, 'time_signature': 4}]


[{'danceability': 0.588,
  'energy': 0.869,
  'key': 6,
  'loudness': -1.998,
  'mode': 1,
  'speechiness': 0.322,
  'acousticness': 0.22,
  'instrumentalness': 0,
  'liveness': 0.585,
  'valence': 0.529,
  'tempo': 91.128,
  'type': 'audio_features',
  'id': '0ElQWopuhQ10wgGouWSVlR',
  'uri': 'spotify:track:0ElQWopuhQ10wgGouWSVlR',
  'track_href': 'https://api.spotify.com/v1/tracks/0ElQWopuhQ10wgGouWSVlR',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0ElQWopuhQ10wgGouWSVlR',
  'duration_ms': 286904,
  'time_signature': 4}]

## Extract the Audio Features of the Requested Song in a Dataframe

In [248]:
print(sp.audio_features(results["tracks"]["items"][0]["id"]))

[{'danceability': 0.588, 'energy': 0.869, 'key': 6, 'loudness': -1.998, 'mode': 1, 'speechiness': 0.322, 'acousticness': 0.22, 'instrumentalness': 0, 'liveness': 0.585, 'valence': 0.529, 'tempo': 91.128, 'type': 'audio_features', 'id': '0ElQWopuhQ10wgGouWSVlR', 'uri': 'spotify:track:0ElQWopuhQ10wgGouWSVlR', 'track_href': 'https://api.spotify.com/v1/tracks/0ElQWopuhQ10wgGouWSVlR', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0ElQWopuhQ10wgGouWSVlR', 'duration_ms': 286904, 'time_signature': 4}]


In [249]:
requested_song_audio_features = pd.DataFrame(sp.audio_features(song_id))
requested_song_audio_features = requested_song_audio_features[["danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo","id","duration_ms"]]

requested_song_audio_features

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms
0,0.588,0.869,-1.998,0.322,0.22,0,0.585,0.529,91.128,0ElQWopuhQ10wgGouWSVlR,286904


In [250]:
simple_requested_song_audio_features = requested_song_audio_features[["danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo","duration_ms"]]

simple_requested_song_audio_features


,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.588,0.869,-1.998,0.322,0.22,0,0.585,0.529,91.128,286904


### 4- Load the StandardScaler using Pickle and use it to scale the REQUESTED_SONG.

#### Load

In [251]:
# Définir le nom du fichier contenant le scaler
nom_fichier_scaler = "scaler.pkl"

# Charger le scaler à partir du fichier
with open(nom_fichier_scaler, 'rb') as fichier:
    scaler = pickle.load(fichier)


#### Scaling the REQUESTED_SONG

In [252]:
print(scaler.get_feature_names_out())

['danceability' 'energy' 'loudness' 'speechiness' 'acousticness'
 'instrumentalness' 'liveness' 'valence' 'tempo' 'duration_ms']


In [253]:
requested_song_scaled = scaler.transform(simple_requested_song_audio_features)
requested_song_scaled

array([[-0.7099237 ,  1.34216406,  1.78830678,  1.59270021, -0.18006988,
        -0.36789564,  3.27048853,  0.19663901, -1.07336966,  1.15327022]])

In [254]:
# scaler_feature_names = scaler.get_feature_names_out()

# # Check if the feature names in REQUESTED_SONG match scaler_feature_names
# if set(REQUESTED_SONG.columns) != set(scaler_feature_names):
# raise ValueError("The feature names in REQUESTED_SONG do not match those used during fit.")

### 5- Use kmeans_model.predict (on the new scaled audio dataset for the REQUESTED_SONG) to predict the cluster (label) for the REQUESTED_SONG.

In [255]:
# cluster_label = kmeans_final.predict(requested_song_scaled)
# cluster_label

In [256]:
requested_song_cluster_label = kmeans_final.predict(requested_song_scaled)
requested_song_cluster_label

array([0])

## 6- Return a random song from the same cluster that the REQUESTED_SONG belongs to from your data frame and suggest it to the user.


In [275]:
# cluster_labels = kmeans_final.predict(X_pca)
# data['cluster_label'] = cluster_labels

songs_same_cluster = data_normalized[data_normalized['cluster'] == requested_song_cluster_label[0]]
display(songs_same_cluster)

# import random




# random_song = songs_same_cluster.sample(n=1, random_state=42)

# suggested_song = random_song['id'].values[0]  # Assuming the song name is in the 'song_name' column
# # print("Suggested song:", suggested_song)


,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,cluster,id
35,1.681924,0.088070,-0.406399,-0.749348,-0.762856,-0.324412,0.713019,1.486690,0.371181,-0.080619,0,66gDy0p06Ev9gk27BFhSLi
48,1.234783,-0.412417,0.212726,-0.742664,0.037864,-0.367896,-0.585329,1.543733,-0.002055,-0.395571,0,44TzvSZLIkM7Y7MEnrvkIl
60,-0.355050,1.526251,0.575579,-0.766894,-0.932318,0.013838,-0.724185,2.114164,-1.115369,1.547361,0,2VohZBjF7cVEcy9vSP74RI
62,-0.000177,1.020012,0.892213,0.389507,0.575184,-0.367887,-0.369591,1.012794,-0.967257,-1.477137,0,4JcfDrtQWE45N1X4MkcbiV
84,0.957982,0.945226,-0.005261,-0.747677,1.300379,-0.367896,-0.586113,2.127328,0.234442,-0.667882,0,1BLXxFPDL2BT37nHKD7KrA
...,...,...,...,...,...,...,...,...,...,...,...,...
2111,1.078639,1.008506,1.358885,-0.681668,-0.206372,-0.367896,0.705174,0.990854,0.929845,-0.494332,0,1N8moY6H0eThviK71VhTzY
2116,0.021116,0.461997,0.233766,0.514840,-0.537031,-0.367896,1.215099,1.504242,-1.031482,0.569983,0,5rf3KdurcBMEOg2LmEHRd7
2123,0.546329,0.513772,1.307838,0.088709,-0.989019,-0.367015,-0.110706,0.411648,-1.672695,-1.246494,0,2jRmKc69qzGKzGU9SdH5u7
2126,0.737960,0.904957,0.344140,-0.395075,-0.679064,-0.367896,0.407064,1.276070,0.112384,-1.070504,0,3w1WjD2zJqjBjDz5fwqQPJ


In [276]:
random_song = songs_same_cluster.sample(n=1)
display(random_song)

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,cluster,id
1199,1.667729,1.388186,1.002586,1.325324,-0.555818,-0.367896,-0.600234,0.380932,-0.223945,-1.095383,0,4TXFhVqa2FWrDICNJkeVyM


In [277]:
random_song_id = random_song['id'].values[0]
print(random_song_id)

4TXFhVqa2FWrDICNJkeVyM


In [278]:
play_song(random_song_id)

In [ ]:
#  For project clustering steps:
# 1- Load the Pandas data frame songs of the audio features (at least 1000 songs as different as possible).🎹 
# 2- Standardize the data using Standardscaler
# 3- Save the scaler for future use on the new user input song, using Pickle. (tomorrow)
# 4- Select the number of clusters k.
# 5- Adjust k means cluster.
# 6- Build the elbow graph to find the best k. (tomorrow)
# 7- Use the model with the best k to assign each observation in your data frame to its cluster number using model.predict (add cluster columns to the pandas data frame).
# 8- Save the model with the best k as your final model using pickle. (tomorrow)

# 📍 For the final product tasks and the full scenario:
# 1- Get the song name from the user as input ---------- DONE
# 2- Play the input song in the music embed player. ---------- DONE
# 3- Get the audio features for this song from the Spotify API using sp.audio_features(trackid). Be careful to keep only the audio features columns.---------- DONE
# 4- Load the StandardScaler using Pickle and use it to scale the REQUESTED_SONG.
# 5- Use kmeans_model.predict(new scaled audio dataset for the REQUESTED_SONG) to predict the cluster (label) for the REQUESTED_SONG.
# 6- Return a random song from the same cluster that the REQUESTED_SONG belongs to from your data frame and suggest it to the user.
# 7- Play it using the built-in music player.
# 8*- Try to Streamlit? ☀️ (optional) 